In [1]:
# train a Deep Learning model on MNIST Data to recognize digits in images using pytorch
# and compute the carbon footprint of the training process

In [2]:
# import libraries
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from codecarbon import EmissionsTracker

In [3]:
# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
training_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [5]:
# set the batch size
batch_size = 64

In [6]:
# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [7]:
# creating model
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [8]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
# optimizer and loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
# import the tracker using explicit mode
tracker = EmissionsTracker()
tracker.start()

epochs = 50
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

emissions: float = tracker.stop()
print(emissions)

[codecarbon INFO @ 12:50:47] [setup] RAM Tracking...
[codecarbon INFO @ 12:50:47] [setup] GPU Tracking...
[codecarbon INFO @ 12:50:47] No GPU found.
[codecarbon INFO @ 12:50:47] [setup] CPU Tracking...
[codecarbon WARNING @ 12:50:47] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 12:50:47] CPU Model on constant consumption mode: Apple M1
[codecarbon INFO @ 12:50:47] >>> Tracker's metadata:
[codecarbon INFO @ 12:50:47]   Platform system: macOS-14.0-arm64-arm-64bit
[codecarbon INFO @ 12:50:47]   Python version: 3.8.18
[codecarbon INFO @ 12:50:47]   CodeCarbon version: 2.3.1
[codecarbon INFO @ 12:50:47]   Available RAM : 16.000 GB
[codecarbon INFO @ 12:50:47]   CPU count: 8
[codecarbon INFO @ 12:50:47]   CPU model: Apple M1
[codecarbon INFO @ 12:50:47]   GPU count: None
[codecarbon INFO @ 12:50:47]   GPU model: None


Epoch 1
-------------------------------
loss: 2.267008  [   64/60000]
loss: 2.266788  [ 6464/60000]
loss: 2.265747  [12864/60000]
loss: 2.244746  [19264/60000]
loss: 2.247543  [25664/60000]
loss: 2.244923  [32064/60000]
loss: 2.232525  [38464/60000]
loss: 2.249650  [44864/60000]
loss: 2.226334  [51264/60000]
loss: 2.216695  [57664/60000]


[codecarbon INFO @ 12:50:53] Energy consumed for RAM : 0.000025 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 12:50:53] Energy consumed for all CPUs : 0.000021 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 12:50:53] 0.000046 kWh of electricity used since the beginning.


Test Error: 
 Accuracy: 64.3%, Avg loss: 2.214514 

Epoch 2
-------------------------------
loss: 2.211700  [   64/60000]
loss: 2.207074  [ 6464/60000]
loss: 2.213621  [12864/60000]
loss: 2.168016  [19264/60000]
loss: 2.184093  [25664/60000]
loss: 2.177958  [32064/60000]
loss: 2.151801  [38464/60000]
loss: 2.183007  [44864/60000]
loss: 2.144310  [51264/60000]
loss: 2.126981  [57664/60000]
Test Error: 
 Accuracy: 66.0%, Avg loss: 2.123357 

Epoch 3
-------------------------------
loss: 2.121545  [   64/60000]
loss: 2.106929  [ 6464/60000]
loss: 2.125800  [12864/60000]
loss: 2.041208  [19264/60000]
loss: 2.074773  [25664/60000]
loss: 2.061812  [32064/60000]
loss: 2.014783  [38464/60000]
loss: 2.066048  [44864/60000]
loss: 2.002576  [51264/60000]
loss: 1.972053  [57664/60000]
Test Error: 
 Accuracy: 67.0%, Avg loss: 1.963931 

Epoch 4
-------------------------------
loss: 1.965406  [   64/60000]


[codecarbon INFO @ 12:51:03] Energy consumed for RAM : 0.000025 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 12:51:03] Energy consumed for all CPUs : 0.000021 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 12:51:03] 0.000046 kWh of electricity used since the beginning.


...

[codecarbon INFO @ 12:54:23] Energy consumed for RAM : 0.000375 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 12:54:23] Energy consumed for all CPUs : 0.000313 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 12:54:23] 0.000688 kWh of electricity used since the beginning.


loss: 0.396731  [57664/60000]
Test Error: 
 Accuracy: 92.0%, Avg loss: 0.278008 

Epoch 50
-------------------------------
loss: 0.238108  [   64/60000]
loss: 0.260091  [ 6464/60000]
loss: 0.184023  [12864/60000]
loss: 0.319910  [19264/60000]
loss: 0.221193  [25664/60000]
loss: 0.302247  [32064/60000]
loss: 0.185052  [38464/60000]
loss: 0.359321  [44864/60000]
loss: 0.296457  [51264/60000]
loss: 0.394279  [57664/60000]


[codecarbon INFO @ 12:54:29] Energy consumed for RAM : 0.000367 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 12:54:29] Energy consumed for all CPUs : 0.000306 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 12:54:29] 0.000673 kWh of electricity used since the beginning.


Test Error: 
 Accuracy: 92.1%, Avg loss: 0.276089 

Done!
0.0002294739190616119


[codecarbon INFO @ 12:54:38] Energy consumed for RAM : 0.000400 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 12:54:39] Energy consumed for all CPUs : 0.000333 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 12:54:39] 0.000734 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:54:54] Energy consumed for RAM : 0.000425 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 12:54:54] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 12:54:54] 0.000779 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:55:09] Energy consumed for RAM : 0.000450 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 12:55:09] Energy consumed for all CPUs : 0.000375 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 12:55:09] 0.000825 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:55:24] Energy consumed for RAM : 0.000475 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 12:55:24] Energy consumed for all CPUs : 0.000396 kWh. Total CPU Power : 5.0 W
[codecarbon